In [1]:
# Import du paquet PythonMIP et de toutes ses fonctionnalités
from mip import *
# Import du paquet time pour calculer le temps de résolution
import time 
import InstanceReader

In [2]:
def solve(data, L, K):
    # Création du modèle vide 
    model = Model(name = "MTZ-EAF", sense = mip.MAXIMIZE, solver_name="CBC")
    
    #### Création des variables y, u et t ####
    y=[]
    u=[]
    for i in range(data.id_max+1):
        y.append([])
        for j in range(data.id_max+1):
            y[i].append(model.add_var(name="y(" + str(i) + "," + str(j) + ")", var_type=BINARY))

    for k in range(len(data.P)):
        u.append([])
        for i in range(data.id_max+1):
            u[k].append([])
            for j in range(data.id_max+1):
                u[k][i].append(model.add_var(name="u("+str(k) + "," + str(i) + "," + str(j)+ ")", var_type=BINARY))
    
    t = [model.add_var(name="t(" + str(i)  + ")", lb=0, ub=(L-1) , var_type=CONTINUOUS) for i in range(data.id_max+1)]
    #On a fait la contrainte 21 : ti <= L-1 pour tout i de V    
    
    
    #### fonction objectif ####
    model += (xsum(data.cost[i][j] * y[i][j]  for i in (data.V) for j in (data.allPatients[int(data.index[int(i)])]))
             + xsum(xsum(data.cost[i][j]*u[k][i][j] for i in (data.V) for j in (data.allPatients[int(data.index[int(i)])])) for k in range (len(data.P))))
    
    
    #### Les contraintes ####
    
    # Contrainte 17
    for k in range(len(data.P)):
        for i in (data.P):
            model += (xsum(u[k][i][j] for j in data.allPatients[int(data.index[int(i)])])
                      -xsum(u[k][int(data.allDonorsInP[i][j])][i] for j in range(1,int(data.allDonorsInP[i][0])+1)) ==0 ) 
    
    #Contrainte 18
    for i in (data.P):
        model += (xsum(y[i][j] for j in data.allPatients[int(data.index[int(i)])])
                      -xsum(y[int(data.allDonors[i][j])][i] for j in range(1,int(data.allDonors[i][0])+1)) ==0 ) 
    
    #Contrainte 19
    #En cours
    for i in (data.P):
        model += (xsum(y[i][j] for j in data.allPatients[int(data.index[int(i)])])
                  + xsum(xsum(u[k][i][j] for j in data.allPatients[int(data.index[int(i)])]) for k in range (len(data.P))) )
    
    #Contrainte 20
    for i in (data.N):
        model += (xsum(y[i][j] for j in data.allPatients[int(data.index[int(i)])]) <=1 )
        
    #Contrainte 21
    for i in range(len(data.P)):
        model += (xsum(u[k][i][j] for i in (data.V) for j in (data.allPatients[int(data.index[int(i)])])) <= K)
        
    #Contrainte 23
    for i in range(len(data.P)):
        for k in range(i):
            model += (xsum(u[k][i][j] for j in (data.allPatients[int(data.index[int(i)])]))
                      - xsum(u[k][k][j] for j in (data.allPatients[int(data.index[int(k)])])) <= 0)
            
    #Contrainte 24
    for i in range(len(data.P)):
        for k in range(i+1,len(data.P)):
            model += (xsum(u[k][i][j] for j in (data.allPatients[int(data.index[int(i)])])) ==0)
    
    #Contrainte 25
    for i in (data.V):
        for j in (data.allPatients[int(data.index[int(i)])]):
            model += (t[i] - t[j] + (len(data.P)+2)* y[i][j] <= len(data.P)+1)
            
    #Contrainte 26:
    for i in (data.N):
        model += (t[i]==0)
                  
    # Limitation du nombre de processeurs
    model.threads = 1  
    # Lancement du chronomètre
    start = time.perf_counter()
    # Résolution du modèle
    status = model.optimize(max_seconds=120)  # temps limite = 120s
    # Arrêt du chronomètre et calcul du temps de résolution
    runtime = time.perf_counter() - start
    
    print("\n----------------------------------")
    if status == OptimizationStatus.OPTIMAL:
        print("Status de la résolution: OPTIMAL")
    elif status == OptimizationStatus.FEASIBLE:
        print("Status de la résolution: TEMPS LIMITE et SOLUTION REALISABLE CALCULEE")
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        print("Status de la résolution: TEMPS LIMITE et AUCUNE SOLUTION CALCULEE")
    elif status == OptimizationStatus.INFEASIBLE or status == OptimizationStatus.INT_INFEASIBLE:
        print("Status de la résolution: IRREALISABLE")
    elif status == OptimizationStatus.UNBOUNDED:
        print("Status de la résolution: NON BORNE")
        
    print("Temps de résolution (s) : ", runtime)    
    print("----------------------------------")

    # Si le modèle a été résolu à l'optimalité ou si une solution a été trouvée dans le temps limite accordé
    if model.num_solutions>0:
        print("Valeur de la fonction objectif de la solution calculée : ", round(model.objective_value,2))  # ne pas oublier d'arrondir si le coût doit être entier
        
    

In [3]:
solve(InstanceReader.Data(str(50),str(0)),10,10)

id max :  127


----------------------------------
Status de la résolution: TEMPS LIMITE et SOLUTION REALISABLE CALCULEE
Temps de résolution (s) :  121.457418112
----------------------------------
Valeur de la fonction objectif de la solution calculée :  50.0
